In [1]:
pip install refractml

  Using cached https://files.pythonhosted.org/packages/f9/0d/023d845cf453feb632b08435c91f7e5d050c0df73c5be66bdbbca2f6ba87/refractml-1.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7b/f5/890a0baca17a61c1f92f72b81d3c31523c99bec609e60c292ea55b387ae8/urllib3-1.26.15-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/fc/a4d5a7554e0067677823f7265cb3ae22aed8a238560b5133b58cda252dad/PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/09/d7/8424b1dcaa5b1a2f824fc440aa1c4ef45e0bf6593d11b37962311614f365/mosaic_utils-1.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/e3/898487e5dbeb612054cf2e0c188463acb358167fef749c53c8bb8918cea1/cl

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyspark

Processing /home/mosaic-ai/.cache/pip/wheels/b3/bb/a5/302c9622d3602de5a21831e14edffa65fb36ecb1c71ebd2a7f/pyspark-3.5.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

Uninstalling urllib3-1.26.15:
  Successfully uninstalled urllib3-1.26.15
  Using cached https://files.pythonhosted.org/packages/7b/f5/890a0baca17a61c1f92f72b81d3c31523c99bec609e60c292ea55b387ae8/urllib3-1.26.15-py2.py3-none-any.whl
ERROR: jupyterlab-server 2.25.3 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
pip show urllib3

Name: urllib3
Version: 1.26.15
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: https://urllib3.readthedocs.io/
Author: Andrey Petrov
Author-email: andrey.petrov@shazow.net
License: MIT
Location: /tmp/pip_packages
Requires: 
Required-by: requests, refractml, snowflake-connector-python, botocore
Note: you may need to restart the kernel to use updated packages.


In [1]:
from refractml import *


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from snowflake.snowpark.session import Session
import configparser

import warnings
warnings.filterwarnings("ignore")

config = configparser.ConfigParser()
config.read("snowflake_connection_new.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [14]:
sales_bangalore_2022 = session.table("SALES_MELBOURNE_COMBINED").to_pandas()

SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: 255002: Optional dependency: 'pandas' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

In [12]:
sales_bangalore_2022.head()

AttributeError: Table object has no attribute head

In [8]:
train_final = pd.read_csv('/data/train_data_model.csv')
train_final.head(1)

,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,PRD0002-OL10330,0.6,0.004323,0.000404,1,0.000465


In [9]:
master_data = sales_bangalore_2022.copy()

In [10]:
master_data = sales_bangalore_2022.copy()
master_data['trans_date'] = pd.to_datetime(master_data['trans_date'],format = '%m/%d/%Y')
master_data['start_date'] = pd.to_datetime(master_data['start_date'],format = '%m/%d/%Y')
master_data['mnth_code'] = pd.to_numeric(master_data['month_code'],errors ='coerce').astype(int)
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [12]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

array([[0.66778068, 0.33221932],
       [0.8336751 , 0.1663249 ],
       [0.44592937, 0.55407063],
       ...,
       [0.66800724, 0.33199276],
       [0.83363294, 0.16636706],
       [0.6676696 , 0.3323304 ]])

In [13]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.60      0.70     30799
           1       0.57      0.82      0.68     20399

    accuracy                           0.69     51198
   macro avg       0.70      0.71      0.69     51198
weighted avg       0.73      0.69      0.69     51198



In [14]:
test_final = pd.read_csv('/data/test_set_model.csv')

In [15]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

(189082, 3) (189082,)


,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity
0,PRD0002-OL10330,0.666667,0.004323,0.000404,1,0.001560,0.807117
1,PRD0002-OL10346,0.166667,0.001430,0.000135,0,0.001560,0.298748
2,PRD0002-OL10347,0.166667,0.001767,0.000168,0,0.001559,0.298690
3,PRD0002-OL10358,1.000000,0.015771,0.001481,1,0.001560,0.950099
4,PRD0002-OL10386,0.833333,0.008417,0.000791,1,0.001560,0.899377
...,...,...,...,...,...,...,...
189077,PRD0168-OL97586,0.166667,0.001948,0.000303,1,0.001560,0.298657
189078,PRD0168-OL97590,0.166667,0.000000,0.000000,0,0.001560,0.298999
189079,PRD0168-OL97602,0.000000,0.001581,0.000242,0,0.001560,0.165792
189080,PRD0168-OL97615,0.333333,0.001319,0.000202,1,0.001560,0.477312


In [16]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity,prediction
0,PRD0002-OL10330,0.666667,0.004323,0.000404,1,0.001560,0.807117,1
1,PRD0002-OL10346,0.166667,0.001430,0.000135,0,0.001560,0.298748,0
2,PRD0002-OL10347,0.166667,0.001767,0.000168,0,0.001559,0.298690,0
3,PRD0002-OL10358,1.000000,0.015771,0.001481,1,0.001560,0.950099,1
4,PRD0002-OL10386,0.833333,0.008417,0.000791,1,0.001560,0.899377,1
...,...,...,...,...,...,...,...,...
189077,PRD0168-OL97586,0.166667,0.001948,0.000303,1,0.001560,0.298657,0
189078,PRD0168-OL97590,0.166667,0.000000,0.000000,0,0.001560,0.298999,0
189079,PRD0168-OL97602,0.000000,0.001581,0.000242,0,0.001560,0.165792,0
189080,PRD0168-OL97615,0.333333,0.001319,0.000202,1,0.001560,0.477312,1


In [17]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77    124905
           1       0.56      0.61      0.58     64177

    accuracy                           0.70    189082
   macro avg       0.67      0.68      0.68    189082
weighted avg       0.71      0.70      0.71    189082



In [18]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [19]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [20]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

,unique_id,prediction,SOQ,OOS
0,PRD0002-OL10330,1,2.714286,1
1,PRD0002-OL10346,0,1.500000,0
2,PRD0002-OL10347,0,1.250000,0
3,PRD0002-OL10358,1,7.166667,1
4,PRD0002-OL10386,1,4.090909,1


In [21]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

In [22]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [23]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

,unique_id,ms_flag,soq,oos_flag,product,outlet
0,PRD0002-OL10330,1,2.714286,1,PRD0002,OL10330
1,PRD0002-OL10346,0,1.500000,0,PRD0002,OL10346
2,PRD0002-OL10347,0,1.250000,0,PRD0002,OL10347
3,PRD0002-OL10358,1,7.166667,1,PRD0002,OL10358
4,PRD0002-OL10386,1,4.090909,1,PRD0002,OL10386


In [24]:
ms_reco=session.createDataFrame(
        ms_reco.values.tolist(),
        schema=ms_reco.columns.tolist())

In [25]:
ms_reco.write.mode("overwrite").save_as_table("ASSORTMENT_PLANNING.PUBLIC.PREDICTION_TABLE")

In [40]:
req = requests.Request()
req.json = {"payload":X_test_unseen.head(1).to_json()}
print({'payload': payload})

{'payload': {'unique_id': 'PRD0002-OL12565', 'frequencym': 0.0, 'sales_value_avg': 0.0, 'sales_units_avg': 0.0, 'sales_indicator': 0, 'scheme_amount_perproduct': 0.001559427, 'probablity': 0.16597489859674663, 'prediction': 0, 'OOS': 0}}


In [41]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.DataFrame(model.predict(data))
    return prediction.to_json()

In [42]:
print(score(loaded_model, req))

{"0":{"0":1.0}}


In [43]:
from refractml import *
from refractml.constants import MLModelFlavours


In [44]:
tmp = register_model(loaded_model, 
               score, 
               name="Assortment_Planning_Prediction", 
               description="prediction of assortment across retailers by product",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               y_true=y_test,
               y_pred=y_pred, 
               prob=probs, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True, 
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['No Failure',' or Failure'],
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
